In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# Remove index_col if there's no extra column or index column at the starting
df_common=pd.read_csv("/content/drive/MyDrive/gene_ext/ins/01/common_wd1.csv", index_col="Unnamed: 0")
df_protein_file = pd.read_csv("/content/drive/MyDrive/gene_ext/ind_protein/ind_chr1_prot.csv")

In [ ]:
# df_common.columns

In [ ]:
# Add column as Length of variations (LOV) in common_wd1.csv
df_common["LOV"]=abs(df_common["endpos.q"]-df_common["startpos.q"])

In [ ]:
df_common.shape

(1270, 13)

In [ ]:
df_common.to_csv("common1_LOV.csv")

In [ ]:
df_protein_file.shape

(578, 11)

In [ ]:
def map_keywords(str_sentence):
    list_keywords = [
        "immunoglobulin","immunoreceptor","autoimmune","TLR","IgG",
        "autoimmune","autophagy","immunogen","immune","innate","T-cell","NF-kappa", "antigen",
        "B-cell","lymphocyte","histocompatibility","CD24","CD4","LY96", "BCR",
        "IFIT3","PGLYRP1","NKG2D","UL16","leukocyte","cytokine", "interleukin","interferon"
        "antimicrobial peptide","beta-defensin 2","IL16","IL2","chemokine", "antibody"
    ]
    bool_found = bool(
        [
            i for i in list_keywords if i.lower() in str_sentence.lower()
        ]
    )
    return bool_found

In [ ]:
# Add column as keyword denotes immunity term if it is true
df_protein_file["keyword_found"] = df_protein_file.apply(lambda row: map_keywords(row["Protein Name"]),axis=1)

In [ ]:
df_protein_file[df_protein_file["keyword_found"]==True].shape

(5, 12)

In [ ]:
df_protein_file[df_protein_file["keyword_found"]==True].drop_duplicates().shape

(5, 12)

In [ ]:
# df_protein_file_imm=df_protein_file[df_protein_file["keyword_found"]==True].drop_duplicates()

In [ ]:
# Add column as start of next gene to find variation in intergenic region
df_protein_file["start_of_next_gene"] = df_protein_file["Start"].shift(-1).fillna(0)

In [ ]:
# df_protein_file.head()

In [ ]:
# original protein data with keyword and start of next gene
df_protein_file.to_csv("protein1.csv")

In [ ]:
a=0
b=0
c=0
d=0
e=0
f=0
g=0
h=0

In [ ]:
from tqdm import tqdm

In [ ]:
# Identifying the variations
final_df = pd.DataFrame(columns=list(df_protein_file.columns)+["gene_type"]+["startpos.q"]+["endpos.q"]+["seq.r"]+["seq.q"]+["LOV"])
for i, row in tqdm(df_common.iterrows()):
    # Check for the at start
    if df_protein_file[(df_protein_file["Start"]==row["startpos.q"]) & (df_protein_file["Stop"]>row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]==row["startpos.q"]) & (df_protein_file["Stop"]>row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "at_start"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        a=a+1
    # check for the at end
    elif df_protein_file[(df_protein_file["Start"]<row["startpos.q"]) & (df_protein_file["Stop"]==row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]<row["startpos.q"]) & (df_protein_file["Stop"]==row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "at_end"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        b=b+1
    # Check for the within gene
    elif df_protein_file[(df_protein_file["Start"]<row["startpos.q"]) & (df_protein_file["Stop"]>row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]<row["startpos.q"]) & (df_protein_file["Stop"]>row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "within_gene"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        c=c+1
    # Check for the near start gene with integenic
    elif df_protein_file[(df_protein_file["Start"]>row["startpos.q"]) & (df_protein_file["Start"]<row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]>row["startpos.q"]) & (df_protein_file["Start"]<row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "near_start_gene with intergenic"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        d=d+1
    # Check for the near stop gene with intergenic
    elif df_protein_file[(df_protein_file["Stop"]>row["startpos.q"]) & (df_protein_file["Stop"]<row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Stop"]>row["startpos.q"]) & (df_protein_file["Stop"]<row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "near_stop_gene with intergenic"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        e=e+1
    # Check for the at start with intergenic gene
    elif df_protein_file[(df_protein_file["Start"]>row["startpos.q"]) & (df_protein_file["Start"]==row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]>row["startpos.q"]) & (df_protein_file["start"]==row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "at_start with intergenic_gene"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        f=f+1
    # check for the at end with intergenic
    elif df_protein_file[(df_protein_file["Stop"]==row["startpos.q"]) & (df_protein_file["Stop"]<row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Stop"]==row["startpos.q"]) & (df_protein_file["Stop"]<row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "at_end with intergenic_gene"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        g=g+1
    # Check for the intergenic gene
    elif df_protein_file[(df_protein_file["Stop"]<row["startpos.q"]) & (df_protein_file["start_of_next_gene"]>row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Stop"]<row["startpos.q"]) & (df_protein_file["start_of_next_gene"]>row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "intergenic_gene"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        h=h+1
    else:
        pass


1270it [00:14, 84.90it/s]


In [ ]:
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)

0
0
491
0
0
0


In [ ]:
final_df.shape

(1730, 19)

In [ ]:
final_df.drop_duplicates().shape

(1730, 19)

In [ ]:
# contasub all gene information has variation including isoform, same locus may have same variation due to isoforms
final_df.to_csv("genesv1.csv")

In [ ]:
# find variation not present in gene and intergene region called as other non coding region
final_df1 = pd.DataFrame(columns=df_common.columns)
for index, row in tqdm(df_common.iterrows()):
    # Check if 'startpos.q' is present in any row of file2
    if row['startpos.q'] not in final_df['startpos.q'].values:
        # Append the row to the DataFrame if not found
        final_df1 = final_df1.append(row, ignore_index=True)

# Save the DataFrame with rows not found in file2 to a new CSV file
final_df1.to_csv("non_coding1.csv", index=False)

0it [00:00, ?it/s]<ipython-input-25-f0850e36b33f>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df1 = final_df1.append(row, ignore_index=True)
1270it [00:00, 6030.35it/s]


In [ ]:
final_df1.shape

(5, 13)

In [ ]:
final_df1.drop_duplicates().shape

(5, 13)

In [ ]:
# extract only the immune genes
imm = pd.DataFrame
imm=final_df[final_df["keyword_found"]==True]

In [ ]:
imm.shape

(12, 19)

In [ ]:
imm.drop_duplicates().shape

(12, 19)

In [ ]:
imm.drop_duplicates().to_csv("immv1.csv")

In [ ]:
# counting the values
# Create a new DataFrame with unique combinations of 'Locus', 'startpos.q', and 'endpos.q'
# this file has unique variation, may missed out some isoforms that has same variations
locus_counts = final_df.drop_duplicates(subset=['Locus', 'startpos.q', 'endpos.q'])

In [ ]:
locus_counts.shape

(1268, 19)

In [ ]:
locus_counts.to_csv("locus_counts1.csv")

In [ ]:
# Extract the variation presents in gene and intergene seperately
locus_count_within_gene=locus_count[locus_count["gene_type"]=="within_gene"]
locus_count_inter_gene=locus_count[locus_count["gene_type"]=="intergenic_gene"]

# extract the variations among genomic regions including gene and intergene has greater the 50 LOV (length of varaition)
locus_LOV=loc_count[loc_count["LOV"]>=50]

In [ ]:
locus_count_within_gene.shape

In [ ]:
locus_count_inter_gene.shape

In [ ]:
locus_LOV.shape

In [ ]:
locus_count_within_gene.to_csv("locus_within_gene20.csv")
locus_count_inter_gene.to_csv("locus_inter_gene20.csv")
locus_LOV.to_csv("locus_50LOV20.csv")

In [ ]:
# count the unique locus has variation. find how many locus has variation
loc1=locus_count_within_gene["Locus"].unique()

In [ ]:
loc1.shape

In [ ]:
# counting the values for immune genes
# Create a new DataFrame with unique combinations of 'Locus', 'startpos.q', and 'endpos.q', only has immune genes
# this file has unique variation, may missed out some isoforms that has same variations
locus_counts_imm = imm.drop_duplicates(subset=['Locus', 'startpos.q', 'endpos.q'])

In [ ]:
locus_counts_imm.shape

(12, 19)

In [ ]:
locus_counts_imm.to_csv("locus_counts_imm1.csv")

In [ ]:
# Extract the variation presents in immune gene and intergene seperately
immloc_count_within_gene=immloc_count[immloc_count["gene_type"]=="within_gene"]
immloc_count_inter_gene=immloc_count[immloc_count["gene_type"]=="intergenic_gene"]

In [ ]:
immloc_count_within_gene.shape

In [ ]:
immloc_count_inter_gene.shape

In [ ]:
immloc_count_within_gene.to_csv("immloc_count_within_gene30.csv")
immloc_count_inter_gene.to_csv("immloc_count_inter_gene30.csv")

In [ ]:
# count the variation based on gene type
gene_count=locus_counts.groupby(["gene_type"]).size().reset_index(name="count")

In [ ]:
gene_count

,gene_type,count
0,intergenic_gene,774
1,within_gene,494


In [ ]:
gene_count.to_csv("gene_count1.csv")

In [ ]:
# extract the immune has how many variations and its gene type
loc_imm_count=locus_counts_imm.groupby(by=["Locus","gene_type"]).size().reset_index(name="count")
# loc_imm_count.columns=["Locus", "gene_type", "count"]

loc_imm_count.to_csv("immlocus_gene_type_counts1.csv", index=False)

In [ ]:
loc_imm_count

,Locus,gene_type,count
0,PGBD5,intergenic_gene,8
1,PGBD5,within_gene,4


In [ ]:
# Calculate value counts
count = locus_counts["gene_type"].value_counts().reset_index()
count.columns = ["gene_type", "Count"]
count.loc[len(count.index)]=["immune_gene", len(immloc_count_within_gene)]
count.loc[len(count.index)]=["non_coding_gene", len(final_df1)]


# Save the result to a CSV file
count.to_csv("subcounts1.csv", index=False)

In [ ]:
import shutil

shutil.copy("protein1.csv","/content/drive/MyDrive/gene_ext/sub/1/new")
shutil.copy("common1_LOV.csv","/content/drive/MyDrive/gene_ext/sub/1/new/")
shutil.copy("genesv1.csv","/content/drive/MyDrive/gene_ext/sub/1/new/")
shutil.copy("immv1.csv","/content/drive/MyDrive/gene_ext/sub/1/new/")
shutil.copy("locus_counts1.csv","/content/drive/MyDrive/gene_ext/sub/1/new/")
shutil.copy("locus_counts_imm1.csv","/content/drive/MyDrive/gene_ext/sub/1/new/")
shutil.copy("immlocus_gene_type_counts1.csv","/content/drive/MyDrive/gene_ext/sub/1/new/")
shutil.copy("gene_count1.csv","/content/drive/MyDrive/gene_ext/sub/1/new/")
shutil.copy("subcounts1.csv","/content/drive/MyDrive/gene_ext/sub/1/new/")
shutil.copy("locus_within_gene20.csv", "/content/drive/MyDrive/gene_ext/sub/20/new")
shutil.copy("locus_inter_gene20.csv", "/content/drive/MyDrive/gene_ext/sub/20/new")
shutil.copy("locus_50LOV20.csv", "/content/drive/MyDrive/gene_ext/sub/20/new")
shutil.copy("non_coding30.csv","/content/drive/MyDrive/gene_ext/sub/30/new/")
shutil.copy("immloc_count_within_gene30.csv","/content/drive/MyDrive/gene_ext/sub/30/new/")
shutil.copy("immloc_count_inter_gene30.csv","/content/drive/MyDrive/gene_ext/sub/30/new/")

'/content/drive/MyDrive/gene_ext/ins/28/new/inscounts28.csv'

In [ ]:
import pandas as pd
sf1=pd.read_csv("/content/drive/MyDrive/gene_ext/sub/11/new/locus_counts11.csv")

In [ ]:
sf1.shape

(2292, 20)

In [ ]:
sf=sf1[sf1["gene_type"]=="at_end with intergenic gene"]

In [ ]:
sf.shape

(1, 20)

In [ ]:
sf

,Unnamed: 0,#Name,Accession,Start,Stop,Strand,GeneID,Locus,Locus tag,Protein product,Length,Protein Name,keyword_found,start_of_next_gene,gene_type,startpos.q,endpos.q,seq.r,seq.q,LOV
107581,143776,chromosome 3,NC_032652.1,98618979,98641766,+,109556870,CDCP2,-,XP_019813808.1,680,PREDICTED: CUB domain-containing protein 2,False,98751341.0,at_start,98618979,98618979,T,A,NaN


In [ ]:
sf1.shape

(17925, 21)

In [ ]:
a=sf1["Locus"].unique()

In [ ]:
a.shape

(304,)

In [ ]:
imf1=pd.read_csv("/content/drive/MyDrive/COW_GENOME_RESEARCH/Objective_1/Whole_Genome_Analysis_Existing_Data/gene_ext/ins/1/new/immloc_count_within_gene.csv")

In [ ]:
imf1

In [ ]:
# extract the immune has how many variations and its gene type
locwithin_imm_count=imf1.groupby(by=["Locus"]).size().reset_index(name="count")

In [ ]:
locwithin_imm_count

,Locus,count
0,ALCAM,6
1,BCL6,2
2,BTLA,1
3,CD80,2
4,CD86,5
5,CD96,3
6,IGSF5,3
7,IL10RB,2
8,IL1RAP,2
9,IL20RB,3


In [ ]:
# Group by 'Locus' and calculate the sum of 'LOV'
grouped = imf1.groupby('Locus')['LOV'].sum().reset_index()
grouped

,Locus,LOV
0,ALCAM,15
1,BCL6,24
2,BTLA,1
3,CD80,5
4,CD86,30
5,CD96,4
6,IGSF5,3
7,IL10RB,7
8,IL1RAP,2
9,IL20RB,8


In [ ]:
# Merge the two files on the 'Locus' column
merged_df = pd.merge(locwithin_imm_count, grouped, on='Locus')

# Rename columns for clarity
merged_df.rename(columns={'count': 'No. of Variations', 'LOV': 'Length of Variations'}, inplace=True)

# Save the result to a new CSV file
merged_df.to_csv("output_file.csv", index=False)

In [ ]:
merged_df

,Locus,No. of Variations,Length of Variations
0,ALCAM,6,15
1,BCL6,2,24
2,BTLA,1,1
3,CD80,2,5
4,CD86,5,30
5,CD96,3,4
6,IGSF5,3,3
7,IL10RB,2,7
8,IL1RAP,2,2
9,IL20RB,3,8


In [ ]:
# create a dataframe to create a table for subetion count among different gene type with immune locus value
table_ins_immcount_1=pd.DataFrame(columns=list(merged_df))

In [ ]:
table_ins_immcount_1

,Locus,No. of Variations,Length of Variations,Chromosome


In [ ]:
merged_df["Chromosome"] = "1"

In [ ]:
table_ins_immcount_1=pd.concat([table_ins_immcount_1, merged_df])

In [ ]:
table_ins_immcount_1

,Locus,No. of Variations,Length of Variations,Chromosome
0,ALCAM,6,15,1
1,BCL6,2,24,1
2,BTLA,1,1,1
3,CD80,2,5,1
4,CD86,5,30,1
5,CD96,3,4,1
6,IGSF5,3,3,1
7,IL10RB,2,7,1
8,IL1RAP,2,2,1
9,IL20RB,3,8,1


In [ ]:
# Reorder columns
reorder_df = table_ins_immcount_1[['Chromosome','Locus', 'No. of Variations', 'Length of Variations']]

In [ ]:
reorder_df

,Chromosome,Locus,No. of Variations,Length of Variations
0,1,ALCAM,6,15
1,1,BCL6,2,24
2,1,BTLA,1,1
3,1,CD80,2,5
4,1,CD86,5,30
5,1,CD96,3,4
6,1,IGSF5,3,3
7,1,IL10RB,2,7
8,1,IL1RAP,2,2
9,1,IL20RB,3,8


In [ ]:
# immune genes information - insertion
import shutil

Chr_list=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, "X"]
# Chr_list = [1]
for chr in Chr_list:
  print(f"chromosome:{chr}")
  in_file1=f"/content/drive/MyDrive/COW_GENOME_RESEARCH/Objective_1/Whole_Genome_Analysis_Existing_Data/gene_ext/ins/{chr}/new/immloc_count_within_gene{chr}.csv"
  outloc=f"/content/drive/MyDrive/COW_GENOME_RESEARCH/Objective_1/Whole_Genome_Analysis_Existing_Data/gene_ext/ins/{chr}/new"
  imf1=pd.read_csv(in_file1)

  # extract the immune has how many variations and its gene type
  locwithin_imm_count=imf1.groupby(by=["Locus"]).size().reset_index(name="count")

  # Group by 'Locus' and calculate the sum of 'LOV'
  grouped = imf1.groupby('Locus')['LOV'].sum().reset_index()

  # Merge the two files on the 'Locus' column
  merged_df = pd.merge(locwithin_imm_count, grouped, on='Locus')

  # Rename columns for clarity
  merged_df.rename(columns={'count': 'No. of Variations', 'LOV': 'Length of Variations'}, inplace=True)

  chr_col=f"{chr}"

  merged_df["Chromosome"] = chr_col

  # Reorder columns
  reorder_df = merged_df[['Chromosome','Locus', 'No. of Variations', 'Length of Variations']]

  out_f1=f"imm_var_count{chr}.csv"
  reorder_df.to_csv(out_f1)

  shutil.copy(out_f1, outloc)


chromosome:1
chromosome:2
chromosome:3
chromosome:4
chromosome:5
chromosome:6
chromosome:7
chromosome:8
chromosome:9
chromosome:10
chromosome:11
chromosome:12
chromosome:13
chromosome:14
chromosome:15
chromosome:16
chromosome:17
chromosome:18
chromosome:19
chromosome:20
chromosome:21
chromosome:22
chromosome:23
chromosome:24
chromosome:25
chromosome:26
chromosome:27
chromosome:28
chromosome:29
chromosome:X
